In [ ]:
!pip install h2o

     |████████████████████████████████| 129.4MB 103kB/s 
  Created wheel for h2o: filename=h2o-3.30.1.3-py2.py3-none-any.whl size=129446676 sha256=4e6762accecd68099ddc192722cefb74d245b6f310424550da59fd98811079fc
  Stored in directory: /root/.cache/pip/wheels/3d/e6/07/53ce9be9cb61b33a79cb3ed1fc39f2dae84f6ee6fe1e373e5e
Successfully built h2o


In [1]:
import pandas as pd

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
df = pd.read_csv("/content/qy_st.csv", index_col=0)

In [ ]:
df = df[['grwtLt','lefLt', 'frmWeek', 'lefCunt', 'lefBt', 'stemThck', 'ph']]#, 'ph', 'cunt', 'otmsuplyqy']]

In [ ]:
df.sample(5)

,grwtLt,lefLt,frmWeek,lefCunt,lefBt,stemThck,ph
3851,0.000000,21.025,36,11.5,24.05,0.0,5.8000
3670,0.000000,0.000,1,0.0,0.00,0.0,5.4875
6977,0.000000,0.000,15,0.0,0.00,0.0,5.5000
6262,16.220833,0.000,1,0.0,0.00,0.0,6.2400
347,0.000000,0.000,42,0.0,0.00,0.0,5.4700


# Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test = train_test_split(df, test_size=0.15, random_state=17)

# H2O AutoML

In [ ]:
import h2o
from h2o.automl import H2OAutoML

In [ ]:
h2o.__version__

'3.30.1.3'

In [ ]:
import multiprocessing
multiprocessing.cpu_count()

2

In [ ]:
import psutil
psutil.virtual_memory()

svmem(total=13653557248, available=10605817856, percent=22.3, used=2991226880, free=6330425344, active=3037155328, inactive=3923812352, buffers=88559616, cached=4243345408, shared=987136, slab=232669184)

In [ ]:
#h2o.init()
h2o.init(ip="127.0.0.1", max_mem_size_GB = 200, nthreads = 10)

Checking whether there is an H2O instance running at http://127.0.0.1:54321 . connected.


H2O_cluster_uptime:,44 mins 41 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.3
H2O_cluster_version_age:,1 month and 4 days
H2O_cluster_name:,H2O_from_python_unknownUser_ty6lx5
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,199.9 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [ ]:
hf = h2o.H2OFrame(x_train)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [ ]:
hf.head()

grwtLt,lefLt,frmWeek,lefCunt,lefBt,stemThck,ph
0,49,38,11.75,39.525,12.285,6
19.075,15.725,36,5,10.95,0,5.76667
0,0,1,0,0,0,5.6
0,0,29,0,0,0,5.8
0,0,36,0,0,0,5.77574
0,0,39,0,0,0,5.47262
0,0,44,0,0,0,6.30833
0,0,16,0,0,0,5.99381
0,0,26,0,0,0,5.8
12.875,28.475,37,8,21.7,0,5.68125


In [ ]:
y_label = "ph"
x_labels = list(df.drop(columns=[y_label]).columns)

In [ ]:
x_labels

['grwtLt', 'lefLt', 'frmWeek', 'lefCunt', 'lefBt', 'stemThck']

In [ ]:
hf[y_label] = hf[y_label]

In [ ]:
aml = H2OAutoML(max_runtime_secs=60*2)

In [ ]:
%%time
aml.train(x = x_labels, y = y_label, training_frame = hf)

AutoML progress: |████████████████████████████████████████████████████████| 100%
CPU times: user 7.18 s, sys: 208 ms, total: 7.39 s
Wall time: 1min 37s


# Leaderboard

In [ ]:
lb = aml.leaderboard

In [ ]:
lb.head(rows=lb.nrows)

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_1_AutoML_20201102_024352,0.0570916,0.238938,0.0570916,0.175546,0.0354917
GBM_4_AutoML_20201102_024352,0.057206,0.239178,0.057206,0.178097,0.0355435
XGBoost_grid__1_AutoML_20201102_024352_model_4,0.0572441,0.239257,0.0572441,0.177585,0.0355376
GBM_2_AutoML_20201102_024352,0.0572468,0.239263,0.0572468,0.17837,0.0355522
GBM_3_AutoML_20201102_024352,0.0573529,0.239485,0.0573529,0.178159,0.0355898
XGBoost_3_AutoML_20201102_024352,0.0575904,0.23998,0.0575904,0.17444,0.0356442
GBM_grid__1_AutoML_20201102_024352_model_10,0.0576442,0.240092,0.0576442,0.176329,0.0356746
XGBoost_grid__1_AutoML_20201102_024352_model_3,0.0577684,0.240351,0.0577684,0.172956,0.0356936
XGBoost_grid__1_AutoML_20201102_024352_model_7,0.0578987,0.240621,0.0578987,0.17308,0.0357391
XGBoost_grid__1_AutoML_20201102_024352_model_2,0.0580433,0.240922,0.0580433,0.180727,0.0358024


In [ ]:
saved_model = h2o.save_model(aml.leader, path = "", force=True)

In [ ]:
saved_model

'/content/GBM_1_AutoML_20201102_024352'

In [9]:
h2o.shutdown

<function h2o.h2o.shutdown(prompt=False)>

In [1]:
import h2o

h2o.init(ip="127.0.0.1", max_mem_size_GB = 200, nthreads = 10)

Checking whether there is an H2O instance running at http://127.0.0.1:54321 . connected.


H2O_cluster_uptime:,3 hours 47 mins
H2O_cluster_timezone:,Asia/Seoul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.3
H2O_cluster_version_age:,2 months and 6 days
H2O_cluster_name:,H2O_from_python_study_pygw46
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,177.5 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [3]:
model = h2o.load_model("./model/total/XGBoost_grid__1_AutoML_PH")

H2OResponseError: Server error water.exceptions.H2OIllegalArgumentException:
  Error: Illegal argument: dir of function: importModel: water.api.FSIOException: FS IO Failure: 
 accessed path : file:/C:/Users/study/Documents/GitHub/Flour_SmartFarm-Platform/4.BE/Flask/./model/total/XGBoost_grid__1_AutoML_PH msg: File not found
  Request: POST /99/Models.bin/
    data: {'dir': './model/total/XGBoost_grid__1_AutoML_PH'}


In [43]:
df = pd.read_csv("qy_grw.csv", index_col=0)
hf = h2o.H2OFrame(df)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [44]:
pred = model.predict(hf)

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [50]:
data_as_df = h2o.as_list(pred)
data_as_df.to_json()

'{"predict":{"0":5.6795406342,"1":5.6795406342,"2":5.6795406342,"3":5.6795406342,"4":5.6795406342,"5":5.6795406342,"6":5.6795406342,"7":5.6642045975,"8":5.6642045975,"9":5.6642045975,"10":5.6642045975,"11":5.6642045975,"12":5.6642045975,"13":5.6642045975,"14":5.5708446503,"15":5.5708446503,"16":5.5708446503,"17":5.5708446503,"18":5.5708446503,"19":5.5708446503,"20":5.5708446503,"21":5.6572284698,"22":5.6572284698,"23":5.6572284698,"24":5.6572284698,"25":5.6572284698,"26":5.6572284698,"27":5.6572284698,"28":5.540532589,"29":5.540532589,"30":5.540532589,"31":5.540532589,"32":5.540532589,"33":5.540532589,"34":5.540532589,"35":5.6464204788,"36":5.6464204788,"37":5.6464204788,"38":5.6464204788,"39":5.6464204788,"40":5.6464204788,"41":5.6464204788,"42":5.5873298645,"43":5.5873298645,"44":5.5873298645,"45":5.5873298645,"46":5.5873298645,"47":5.5873298645,"48":5.5873298645,"49":5.6007022858,"50":5.6007022858,"51":5.6007022858,"52":5.6007022858,"53":5.6007022858,"54":5.6007022858,"55":5.6007022

In [7]:
from flask import request, redirect, render_template, session, Blueprint, jsonify
import pandas as pd
import json

# data = {'haa' : [213], 'ee' : ['QQQQ']}
# {"id":9.0,"frtstGrupp":0.0,"flanGrupp":0.0,"frtstCo":0.0,"frmhsFclu":0.0,"hvstGrupp":0.0,"grwtLt":0.0,"fcluHg":0.0,"lefLt":0.0,"hvstCo":0.0,"frmWeek":43.0,"lefCunt":0.0,"lefBt":0.0,"stemThck":0.0,"outtrn":3140.0,"ec":2.104545442,"ph":6.086363619,"cunt":8.5,"otmsuplyqy":185.0}
# data = json.dumps(data)

# data = json.loads(data)
df = {"id":9.0,"frtstGrupp":0.0,"flanGrupp":0.0,"frtstCo":0.0,"frmhsFclu":0.0,"hvstGrupp":0.0,"grwtLt":0.0,"fcluHg":0.0,"lefLt":0.0,"hvstCo":0.0,"frmWeek":43.0,"lefCunt":0.0,"lefBt":0.0,"stemThck":0.0,"outtrn":3140.0,"ec":2.104545442,"ph":6.086363619,"cunt":8.5,"otmsuplyqy":185.0}
for key, value in df.items():
    df[key] = [value]
df = pd.DataFrame(df)
df

,id,frtstGrupp,flanGrupp,frtstCo,frmhsFclu,hvstGrupp,grwtLt,fcluHg,lefLt,hvstCo,frmWeek,lefCunt,lefBt,stemThck,outtrn,ec,ph,cunt,otmsuplyqy
0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43.0,0.0,0.0,0.0,3140.0,2.104545,6.086364,8.5,185.0


In [ ]:
DataFrame.from_dict